In [ ]:
import os
import gc
import cv2
import time
import random
from monai.inferers import sliding_window_inference

# For data manipulation
import numpy as np
import pandas as pd

# Pytorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp
from pytorch_toolbelt import losses as L

# Utils
from tqdm.auto import tqdm

# For Image Models
import timm
from timm.utils import ModelEmaV2

# Albumentations for augmentations
import albumentations as A
from albumentations.pytorch import ToTensorV2

import warnings
warnings.filterwarnings("ignore")

## using gpu:1
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

def seed_everything(seed=123):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
seed_everything()

In [ ]:
class Customize_Model(nn.Module):
    def __init__(self, model_name, cls):
        super().__init__()
        self.cls= cls
        self.model = timm.create_model(model_name, 
                                       pretrained=True, 
                                       num_classes=self.cls, 
                                       in_chans= 4,
                                       drop_rate= CFG['drop_out'], 
                                       drop_path_rate= CFG['drop_path'])
        
    def forward(self, image):
        x = self.model(image)
        return x
    
    
class Slide_Window_Model(nn.Module):
    def __init__(self, model_name, cls):
        super().__init__()
        self.cls= cls
        self.model = timm.create_model(model_name, 
                                       pretrained=True, 
                                       num_classes=cls, 
                                       in_chans= 4,
                                       drop_rate= CFG['drop_out'], 
                                       drop_path_rate= CFG['drop_path'])
        
    def forward(self, image):
        x = self.model(image)
        return x if self.training else x.view(-1, self.cls, 1, 1)

In [ ]:
def get_train_transform(img_size):
    return A.Compose([
        A.SmallestMaxSize(max_size=img_size, interpolation=3, p=1),
#         A.Resize(img_size, img_size),
        A.RandomCrop(CFG['img_crop'], CFG['img_crop'], p=1),
        
        
        A.HorizontalFlip(p=0.5),
#         A.VerticalFlip(p=0.5),
#         A.Transpose(p=0.5),
#         A.RandomRotate90(p=0.5),
        
#         A.RandomBrightnessContrast(brightness_limit=0.15, contrast_limit=0.15, p=0.5),
#         A.Blur(blur_limit= 3, p=0.3), 
#         A.GaussNoise(p=0.3),
#         A.OneOf([
#             A.GridDistortion(num_steps=5, distort_limit=0.05, p=1.0),
#             A.ElasticTransform(alpha=1, sigma=50, alpha_affine=50, p=1.0)
#         ], p=0.3),
        A.ShiftScaleRotate(shift_limit=0.15, scale_limit=0.05, rotate_limit= 15,
                                        interpolation=cv2.INTER_LINEAR, border_mode=0, p=0.7),
        ToTensorV2(p=1.0),
    ])


def get_test_transform(img_size):
    return A.Compose([
        A.SmallestMaxSize(max_size=img_size, interpolation=3, p=1),
#         A.Resize(img_size, img_size),
        ToTensorV2(p=1.0),
    ])

In [ ]:
class Customize_Dataset(Dataset):
    def __init__(self, df, transforms=None, mixup=False):
        self.df = df
        self.transforms = transforms
        self.mixup= mixup
    
    def mixup_aug(self, img_1, mask_1, 
                        img_2, mask_2):
        """
        img: numpy array of shape (height, width,channel)
        mask: numpy array of shape (height, width,channel)
        """
        ## mixup
        weight= np.random.beta(a=0.5, b=0.5)
        img= img_1*weight + img_2*(1-weight)
        mask= mask_1*weight + mask_2*(1-weight)
        return img.astype(np.uint8), mask
    
    def read_data(self, data):
        def norm_to_255(img):
            img= img-img.min()
            img= img/img.max()
            img= img*255
            return img.astype(np.uint8)
        
        def norm_to_standard(img):
            ep = 1e-6
            m = np.nanmean(img.flatten())
            s = np.nanstd(img.flatten())
            img = (img-m)/(s+ep)
            img = np.nan_to_num(img, nan=0.0)
            return img
        
        raw= pd.read_parquet(data['image_path'])
        
        col= list(raw.filter(like='LL', axis=1))
        img_LL= np.log( np.clip(raw[col].T.values,np.exp(-4),np.exp(8)) )
        col= list(raw.filter(like='RL', axis=1))
        img_RL= np.log( np.clip(raw[col].T.values,np.exp(-4),np.exp(8)) )
        col= list(raw.filter(like='RP', axis=1))
        img_RP= np.log( np.clip(raw[col].T.values,np.exp(-4),np.exp(8)) )
        col= list(raw.filter(like='LP', axis=1))
        img_LP= np.log( np.clip(raw[col].T.values,np.exp(-4),np.exp(8)) )
        
        img= np.array([img_LL, img_RL, img_RP, img_LP]).transpose(1,2,0)
        img= norm_to_standard(img)
        
        label= np.array(eval(data['soft_label']))
        return img, label
    
    def __getitem__(self, index):
        data = self.df.loc[index]
        img, label= self.read_data(data)
        
        # use mixup
        if self.mixup and np.random.rand() >= (1-self.mixup):
            img_1= img
            label_1= np.array(label)
            while True:
                indx= np.random.randint(len(self.df))
                data= self.df.loc[indx]
                img_2, label_2= self.read_data(data)
                if label_1.argmax(0)!=label_2.argmax(0): break
            img, label= self.mixup_aug(img_1, label_1, 
                                       img_2, label_2)
        
        if self.transforms:
            img = self.transforms(image=img)["image"]
            
        return {
            'image': torch.tensor(img, dtype=torch.float32),
            'label': torch.tensor(label, dtype=torch.float32),
        }
    
    def __len__(self):
        return len(self.df)

In [ ]:
class Customize_loss(nn.Module):
    def  __init__(self):
        super().__init__()
        self.CrossEntropy= nn.CrossEntropyLoss(weight= None, label_smoothing=0.)
        self.FocalCosineLoss= L.FocalCosineLoss()
    
    def forward(self, y_pred, y_true):
        loss= 1*self.CrossEntropy(y_pred, y_true)
        return loss

In [ ]:
def train_epoch(dataloader, model, criterion, optimizer, model_ema):
    scaler= amp.GradScaler()
    model.train()

    ep_loss= []
    for i, data in enumerate(tqdm(dataloader)):

        imgs= data['image'].to('cuda')
        labels= data['label'].to('cuda')
        
        with amp.autocast():
            preds= model(imgs)
            loss= criterion(preds, labels)
            ep_loss.append(loss.item())
            loss/= CFG['gradient_accumulation']
            scaler.scale(loss).backward()
            
            if (i+1) % CFG['gradient_accumulation']== 0:
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()
                
            if model_ema: model_ema.update(model)
                
    return np.mean(ep_loss)

In [ ]:
from metrics import *

# def valid_epoch(dataloader, model, criterion):
#     model.eval()
    
#     ep_loss= []
#     all_pred= []
#     all_label= []
#     for i, data in enumerate(tqdm(dataloader)):

#         imgs= data['image'].to('cuda')
#         labels= data['label'].to('cuda')
#         all_label.extend(labels.cpu().numpy())
        
#         with torch.no_grad():
#             preds= model(imgs)
#             loss= criterion(preds, labels)
#             ep_loss.append(loss.item())
#         all_pred.extend(preds.cpu().softmax(dim=-1).numpy())
        
    
#     ## caculate metrics
#     all_label= np.array(all_label).argmax(1)
#     all_pred= np.array(all_pred)
    
#     acc= Accuracy(all_pred, all_label)
#     print(f'accuracy: {acc}')
#     recall= Mean_Recall(all_pred, all_label)
#     print(f'mean_recall: {recall}')
#     auc= AUC(all_pred, all_label)
#     print(f'AUC: {auc}')
    
#     score= auc
#     return np.mean(ep_loss), score


def valid_epoch(dataloader, model, criterion):
    model.eval()
    
    ep_loss= []
    all_pred= []
    all_label= []
    for i, data in enumerate(tqdm(dataloader)):

        imgs= data['image'].to('cuda')
        labels= data['label'].to('cuda')
        all_label.extend(labels.cpu().numpy())
        
        with torch.no_grad():
            preds= sliding_window_inference(imgs, 
                                            roi_size=CFG['img_crop'], 
                                            mode= 'gaussian',
                                            sw_batch_size=4, 
                                            predictor=model)
            preds= preds.view(preds.shape[0],model.cls,-1).max(dim=-1).values
            loss= criterion(preds, labels)
            ep_loss.append(loss.item())
        all_pred.extend(preds.cpu().softmax(dim=-1).numpy())
    
    ## caculate metrics
    soft_label= all_label.copy()
    all_label= np.array(all_label).argmax(1)
    all_pred= np.array(all_pred)
    
    acc= Accuracy(all_pred, all_label)
    print(f'accuracy: {acc}')
    recall= Mean_Recall(all_pred, all_label)
    print(f'mean_recall: {recall}')
    auc= AUC(all_pred, all_label)
    print(f'AUC: {auc}')
    kl_score= kl_divergence(soft_label, all_pred)
    print(f'kl_divergence: {kl_score}')
    
    score= kl_score
    return np.mean(ep_loss), score

# CFG

In [ ]:
CFG= {
    'fold': 0,
    'epoch': 30,
    'model_name': 'tf_efficientnet_b0_ns',
    
    'img_size': 224,
    'img_crop': 224,
    
    'batch_size': 32,
    'gradient_accumulation': 1,
    'gradient_checkpoint': False,
    'drop_out': 0.3,
    'drop_path': 0.2,
    'mixup': 0.,
    'EMA': 0., #0.995
    
    'lr': 3e-4,
    'weight_decay': 0.,
    
    'num_classes': 6,
    'load_model': False,
    'save_model': './train_model'
}

# Prepare Dataset

In [ ]:
df= pd.read_csv('../Data/train_spectrogram.csv')
train_df= df[df['fold']!=CFG['fold']].reset_index(drop=True)
valid_df= df[df['fold']==CFG['fold']].reset_index(drop=True)
print(f'train dataset: {len(train_df)}')
print(f'valid dataset: {len(valid_df)}')

train_dataset= Customize_Dataset(train_df, get_train_transform(CFG['img_size']), mixup=CFG['mixup'])
valid_dataset= Customize_Dataset(valid_df, get_test_transform(CFG['img_size']), mixup=False)

train_loader= DataLoader(train_dataset, batch_size= CFG['batch_size'], shuffle=True, num_workers=0)
valid_loader= DataLoader(valid_dataset, batch_size=1, shuffle=False, num_workers=0)
train_df.head()

# Train

In [ ]:
## create model
if CFG['load_model']:
    print(f"load_model: {CFG['load_model']}")
    model= torch.load(CFG['load_model'], map_location= 'cuda')
else:
    model= Slide_Window_Model(CFG['model_name'], CFG['num_classes'])
    
if CFG['gradient_checkpoint']: 
    print('use gradient checkpoint')
    model.model.set_grad_checkpointing(enable=True)
    
## EMA
model.to('cuda')
if CFG['EMA']:
    print(f"Use EMA: {CFG['EMA']}")
    model_ema= ModelEmaV2(model, decay=CFG['EMA'])
    model_ema.to('cuda')
else:
    model_ema= type('model_ema', (object,), {'module':{}})
    
## hyperparameter
criterion= Customize_loss()
optimizer= optim.AdamW(model.parameters(), lr= CFG['lr'], weight_decay= CFG['weight_decay'])

## start training
best_score= 100000
for ep in range(1, CFG['epoch']+1):
    print(f'\nep: {ep}')
    
    if CFG['EMA']: train_loss= train_epoch(train_loader, model, criterion, optimizer, model_ema)
    else: 
        train_loss= train_epoch(train_loader, model, criterion, optimizer, False)
        model_ema.module= model
    valid_loss, valid_acc= valid_epoch(valid_loader, model_ema.module, criterion)
    print(f'train loss: {round(train_loss, 5)}')
    print(f'valid loss: {round(valid_loss, 5)}, valid_acc: {round(valid_acc, 5)}')
    
    if valid_acc <= best_score:
        best_score= valid_acc
        torch.save(model_ema.module, f"{CFG['save_model']}/cv{CFG['fold']}_best.pth")
        print(f'model save at score: {round(best_score, 5)}')
        
        ## save model every epoch
        torch.save(model_ema.module, f"{CFG['save_model']}/cv{CFG['fold']}_ep{ep}.pth")